<a href="https://colab.research.google.com/github/AWH-GlobalPotential-X/AWH-Geo/blob/main/notebooks/Population_withoutSMDW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This tool calculates the world population without safely managed drinking water (SMDW) based on  official estimates from the JMP of WHO.

This tool requires a [Google Drive](https://drive.google.com/drive/my-drive) and [Earth Engine Account](https://developers.google.com/earth-engine/)

Click "Connect" at the top right of this notebook.

Then run each of the code blocks below, following instructions.

In [ ]:
#@title Basic setup and earthengine access.

print('Welcome to the Population without SMDW tool')

# import, authenticate, then initialize EarthEngine module ee
# https://developers.google.com/earth-engine/python_install#package-import
import ee 
print('Make sure the EE version is v0.1.215 or greater...')
print('Current EE version = v' + ee.__version__)
print('')
ee.Authenticate()
ee.Initialize()

worldGeo = ee.Geometry.Polygon( # Created for some masking and geo calcs
  coords=[[-180,-90],[-180,0],[-180,90],[-30,90],[90,90],[180,90],
          [180,0],[180,-90],[30,-90],[-90,-90],[-180,-90]],
  geodesic=False,
  proj='EPSG:4326'
)



In [ ]:
#@title Click to run calculation

"""
This script creates an image representing the best-estimate distribution of the 
world's 2.2 billion people without SMDW. It applies a weight to a high-resolution
population image based on the % proportion of people without SMDW from JMP 
surveys, created in the JMP_GeoProcessor (on FigShare) and exported as the 
JMP GeoFabric. 

Results are created using two different population images with worldwide coverage:
1) WorldPop 2017 top-down residential population counts (1km res)
2) LandScan 2017 ambient population counts (1km res) from the Oak Ridge Nat'l Lab (not included here)

First, the JMP GeoFabric is converted into an image, then weighted across the pop images.
""" 


jmpGeofabric = ee.FeatureCollection('projects/h2e/publication/jmpGeofabric')

# JMP GeoFabric converted to image
jmpGeofabric_image = jmpGeofabric.reduceToImage(
  properties=['WTRST_PER'], 
  reducer=ee.Reducer.first()
).rename('WTRST_PER')

# JMP GeoFabric adjusted for anomalies in subnational safely managed estimates (see paper)
def adjust(f):
  f = ee.Feature(f)
  SM_SE_adj = ee.Number(f.get('SM_SE')).multiply(0.715).subtract(6.39)
  return ee.Feature(f).setMulti({
    'SM_SE_adj': SM_SE_adj,
    'WTRST_PER': ee.Number(100).subtract(SM_SE_adj)
  })

jmpGeofabric_adj = jmpGeofabric.filterMetadata('REGLEV','equals','SN').map(adjust
      ).merge(jmpGeofabric.filterMetadata('REGLEV','equals','N'))

jmpGeofabric_image_adj = jmpGeofabric_adj.reduceToImage(
  properties=['WTRST_PER'], 
  reducer=ee.Reducer.first()
).rename('WTRST_PER')

# population calcs

# World Pop version
worldPop_2017 = ee.Image('projects/h2e/publication/ppp_2017_1km_Aggregated')
worldPop_2017_scale = worldPop_2017.projection().nominalScale()

# calculate water starved population (100 - jmp_safelyManaged) * population of grid cell
pop_noSMDW = worldPop_2017.multiply(jmpGeofabric_image.select(['WTRST_PER']
  ).divide(100)).int().rename('pop_noSMDW')
# log of above
popLog_noSMDW = pop_noSMDW.add(1).log10()

# Export pop no SMDW WorldPop (default)
task1 = ee.batch.Export.image.toDrive(
  image=ee.Image(pop_noSMDW),
  region=worldGeo,
  scale=worldPop_2017_scale.getInfo(),
  crs='EPSG:4326',
  maxPixels=1e12,
  # folder='', # write Drive folder here if desired
  description='pop_noSMDW',
  fileNamePrefix='pop_noSMDW',
  fileFormat='GeoTIFF'
)
task1.start()

task2 = ee.batch.Export.image.toDrive(
  image=ee.Image(popLog_noSMDW),
  region=worldGeo,
  scale=worldPop_2017_scale.getInfo(),
  crs='EPSG:4326',
  maxPixels=1e12,
  # folder='', # write Drive folder here if desired
  description='popLog_noSMDW',
  fileNamePrefix='popLog_noSMDW',
  fileFormat='GeoTIFF'
)
task2.start()

# # Landscan
# landScan = ee.Image('ORNL/LandScan/2017')
# landScan_scale = landScan.projection().nominalScale()

# # calculate water starved population (100 - jmp_safelyManaged) * population of grid cell
# pop_noSMDW_landScan = landScan.multiply(
#   jmpGeofabric_image.select(['WTRST_PER']).divide(100)).int().rename('pop_noSMDW')
# # log of above
# popLog_noSMDW_landScan = pop_noSMDW_landScan.add(1).log10()

# # Export pop no SMDW landscan
# task3 = ee.batch.Export.image.toDrive(
#   image=ee.Image(pop_noSMDW_landScan),
#   region=worldGeo,
#   scale=landScan_scale.getInfo(),
#   crs='EPSG:4326',
#   maxPixels=1e12,
#   # folder='', # write Drive folder here if desired
#   description='pop_noSMDW_landScan',
#   fileNamePrefix='pop_noSMDW_landScan',
#   fileFormat='GeoTIFF'
# )
# task3.start()

# task4 = ee.batch.Export.image.toDrive(
#   image=ee.Image(popLog_noSMDW_landScan),
#   region=worldGeo,
#   scale=landScan_scale.getInfo(),
#   crs='EPSG:4326',
#   maxPixels=1e12,
#   # folder='', # write Drive folder here if desired
#   description='popLog_noSMDW_landScan',
#   fileNamePrefix='popLog_noSMDW_landScan',
#   fileFormat='GeoTIFF'
# )
# task4.start()

print("Complete")



Go to https://code.earthengine.google.com/ to track task progress.

This will save as GeoTIFF named "pop_noSMDW" and "popLog_noSMDW" in your root [Google Drive folder](https://drive.google.com/drive/my-drive). 